In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("UDF")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/09 20:18:57 INFO SparkContext: Running Spark version 3.5.0
24/09/09 20:18:57 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/09 20:18:57 INFO SparkContext: Java version 11.0.24
24/09/09 20:18:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/09 20:18:57 INFO ResourceUtils: ==============================================================
24/09/09 20:18:57 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/09 20:18:57 INFO ResourceUtils: ==============================================================
24/09/09 20:18:57 INFO SparkContext: Submitted application: UDF
24/09/09 20:18:57 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name:

spark: SparkSession = org.apache.spark.sql.SparkSession@252e7fb0
import spark.implicits._

In [4]:
val data = Seq(
    (1, "Michael", 3000),
    (2, "Andy", 4500),
    (3, "Justin", 3500),
    (4, "Berta", 4000)
).toDF("id", "name", "salary")

data.show()

24/09/09 20:19:01 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/09 20:19:01 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/UDF%20UDAF/spark-warehouse'.
24/09/09 20:19:02 INFO CodeGenerator: Code generated in 266.454145 ms
24/09/09 20:19:03 INFO CodeGenerator: Code generated in 10.865038 ms
24/09/09 20:19:03 INFO CodeGenerator: Code generated in 16.565252 ms


+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|Michael|  3000|
|  2|   Andy|  4500|
|  3| Justin|  3500|
|  4|  Berta|  4000|
+---+-------+------+



data: DataFrame = [id: int, name: string ... 1 more field]

## UDF без аргументов

In [5]:
val randomUDF = udf(() => java.lang.Math.random())
data.withColumn("random", randomUDF()).show()

24/09/09 20:19:04 INFO CodeGenerator: Code generated in 9.33737 ms
24/09/09 20:19:04 INFO CodeGenerator: Code generated in 10.716625 ms
24/09/09 20:19:04 INFO CodeGenerator: Code generated in 13.929473 ms


+---+-------+------+-------------------+
| id|   name|salary|             random|
+---+-------+------+-------------------+
|  1|Michael|  3000|0.04938740049666357|
|  2|   Andy|  4500| 0.8847461491619165|
|  3| Justin|  3500|0.10769906037254529|
|  4|  Berta|  4000| 0.7566436944877213|
+---+-------+------+-------------------+



randomUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell5$Helper$$Lambda$5034/0x00000008418cf840@64d01446,
  DoubleType,
  List(),
  Some(
    ExpressionEncoder(
      BoundReference(0, DoubleType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, DoubleType),
          DoubleType,
          List("- root class: \"double\"")
        ),
        List("- root class: \"double\"")
      ),
      Double
    )
  ),
  None,
  false,
  true
)

## UDF с одним аргументом

In [6]:
val plusOneUDF = udf((x: Int) => x + 1)
data.select($"name", $"salary", plusOneUDF($"salary").as("new_salary")).show()

24/09/09 20:19:05 INFO CodeGenerator: Code generated in 8.276199 ms


+-------+------+----------+
|   name|salary|new_salary|
+-------+------+----------+
|Michael|  3000|      3001|
|   Andy|  4500|      4501|
| Justin|  3500|      3501|
|  Berta|  4000|      4001|
+-------+------+----------+



plusOneUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell6$Helper$$Lambda$5112/0x0000000841901840@5590f1e0,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, IntegerType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, IntegerType),
          IntegerType,
          List("- root class: \"int\"")
        ),
        List("- root class: \"int\"")
      ),
      Int
    )
  ),
  None,
  false,
  true
)

## UDF с двумя аргументами

In [7]:
val strLenPlusUDF = udf((x: String, y: Int) => x.length + y)

strLenPlusUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell7$Helper$$Lambda$5150/0x0000000841917040@6539489e,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        StaticInvoke(
          class org.apache.spark.unsafe.types.UTF8String,
          StringType,
          "fromString",
          List(BoundReference(0, ObjectType(class java.lang.String), true)),
          List(),
          true,
          false,
          true
        ),
        Invoke(
          UpCast(
            GetColumnByOrdinal(0, StringType),
            StringType,
            List("- root class: \"java.lang.String\"")
          ),
          "toString",
          ObjectType(class java.lang.String),
          List(),
          List(),
          true,
          false,
          true
        ),
        java.lang.String
      )
    ),
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetC

In [8]:
data.select($"id", $"name", length($"name"), strLenPlusUDF($"name",  $"id").as("nameLenPlusId")).show()

24/09/09 20:19:06 INFO CodeGenerator: Code generated in 9.387981 ms


+---+-------+------------+-------------+
| id|   name|length(name)|nameLenPlusId|
+---+-------+------------+-------------+
|  1|Michael|           7|            8|
|  2|   Andy|           4|            6|
|  3| Justin|           6|            9|
|  4|  Berta|           5|            9|
+---+-------+------------+-------------+



In [9]:
data.select($"name", length($"name"), strLenPlusUDF($"name", lit(1)).as("nameLenPlus1")).show()

+-------+------------+------------+
|   name|length(name)|nameLenPlus1|
+-------+------------+------------+
|Michael|           7|           8|
|   Andy|           4|           5|
| Justin|           6|           7|
|  Berta|           5|           6|
+-------+------------+------------+



## UDF для выражений WHERE

In [10]:
val oneArgFilterUDF= udf((n: Int) => { n > 2 })
data.where(oneArgFilterUDF($"id")).show()

24/09/09 20:19:07 INFO CodeGenerator: Code generated in 24.78181 ms


+---+------+------+
| id|  name|salary|
+---+------+------+
|  3|Justin|  3500|
|  4| Berta|  4000|
+---+------+------+



oneArgFilterUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell10$Helper$$Lambda$5179/0x000000084192b040@3b4945f3,
  BooleanType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, BooleanType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, BooleanType),
          BooleanType,
          List("- root class: \"boolean\"")
        ),
        List("- root class: \"boolean\"")
      ),
      Boolean
    )
  ),
  None,
  false,
  true
)

## Spark SQL UDF

In [11]:
val cubed = (s: Long) => {s * s * s}

cubed: Long => Long = ammonite.$sess.cell11$Helper$$Lambda$5194/0x0000000841935040@784a2eb4

In [12]:
spark.udf.register("cubed", cubed)

res12: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell11$Helper$$Lambda$5194/0x0000000841935040@784a2eb4,
  LongType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, LongType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, LongType),
            LongType,
            List("- root class: \"long\"")
          ),
          List("- root class: \"long\"")
        ),
        Long
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, LongType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, LongType),
          LongType,
          List("- root class: \"long\"")
        ),
        List("- root class: \"long\"")
      ),
      Long
    )
  ),
  Some("cubed"),
  false,
  true
)

In [13]:
spark.range(1, 10).createOrReplaceTempView("udf_test")

In [14]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

24/09/09 20:19:09 INFO CodeGenerator: Code generated in 37.364227 ms
24/09/09 20:19:09 INFO SparkContext: Starting job: show at cell14.sc:1
24/09/09 20:19:09 INFO DAGScheduler: Got job 0 (show at cell14.sc:1) with 8 output partitions
24/09/09 20:19:09 INFO DAGScheduler: Final stage: ResultStage 0 (show at cell14.sc:1)
24/09/09 20:19:09 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:19:09 INFO DAGScheduler: Missing parents: List()
24/09/09 20:19:09 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[3] at show at cell14.sc:1), which has no missing parents
24/09/09 20:19:10 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 24.7 KiB, free 4.5 GiB)
24/09/09 20:19:10 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 11.3 KiB, free 4.5 GiB)
24/09/09 20:19:10 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ubuntu:41801 (size: 11.3 KiB, free: 4.5 GiB)
24/09/09 20:19:10 INFO SparkContext: Creat

+---+--------+
| id|id_cubed|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
|  9|     729|
+---+--------+



## Производительность

In [15]:
val train = spark.read.option("inferSchema", true).option("header", true).csv("data/train.csv").cache()

24/09/09 20:19:11 INFO InMemoryFileIndex: It took 15 ms to list leaf files for 1 paths.
24/09/09 20:19:11 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/09 20:19:11 INFO FileSourceStrategy: Pushed Filters: 
24/09/09 20:19:11 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#167, None)) > 0)
24/09/09 20:19:11 INFO CodeGenerator: Code generated in 10.238774 ms
24/09/09 20:19:11 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 198.6 KiB, free 4.5 GiB)
24/09/09 20:19:11 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 4.5 GiB)
24/09/09 20:19:11 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on ubuntu:41801 (size: 34.1 KiB, free: 4.5 GiB)
24/09/09 20:19:11 INFO SparkContext: Created broadcast 1 from csv at cell15.sc:1
24/09/09 20:19:11 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 419430

train: Dataset[Row] = [row_id: int, timestamp: bigint ... 8 more fields]

In [16]:
train.show(5)

24/09/09 20:20:16 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 198.6 KiB, free 4.5 GiB)
24/09/09 20:20:16 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 4.5 GiB)
24/09/09 20:20:16 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on ubuntu:41801 (size: 34.1 KiB, free: 4.5 GiB)
24/09/09 20:20:16 INFO SparkContext: Created broadcast 5 from show at cell16.sc:1
24/09/09 20:20:16 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/09/09 20:20:16 INFO DAGScheduler: Got job 3 (show at cell16.sc:1) with 44 output partitions
24/09/09 20:20:16 INFO DAGScheduler: Final stage: ResultStage 3 (show at cell16.sc:1)
24/09/09 20:20:16 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:20:16 INFO DAGScheduler: Missing parents: List()
24/09/09 20:20:16 INFO DAGScheduler: Submitting ResultStage 3 (FileScan csv [row_i

+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       NULL|                          NULL|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [17]:
train.count()

24/09/09 20:22:09 INFO BlockManagerInfo: Removed broadcast_6_piece0 on ubuntu:41801 in memory (size: 7.9 KiB, free: 2.2 GiB)
24/09/09 20:22:09 INFO CodeGenerator: Code generated in 7.828827 ms
24/09/09 20:22:09 INFO DAGScheduler: Registering RDD 27 (count at cell17.sc:1) as input to shuffle 0
24/09/09 20:22:09 INFO DAGScheduler: Got map stage job 5 (count at cell17.sc:1) with 44 output partitions
24/09/09 20:22:09 INFO DAGScheduler: Final stage: ShuffleMapStage 5 (count at cell17.sc:1)
24/09/09 20:22:09 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:22:09 INFO DAGScheduler: Missing parents: List()
24/09/09 20:22:09 INFO DAGScheduler: Submitting ShuffleMapStage 5 (MapPartitionsRDD[27] at count at cell17.sc:1), which has no missing parents
24/09/09 20:22:09 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 23.7 KiB, free 2.2 GiB)
24/09/09 20:22:09 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 11.0 KiB, fre

res17: Long = 101230332L

In [18]:
val to_month = udf((ts_ms: Long) => ts_ms / 31536000000L * 12) // 1 year is 31536000000 ms

to_month: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell18$Helper$$Lambda$7215/0x0000000842005040@24401d55,
  LongType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, LongType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, LongType),
            LongType,
            List("- root class: \"long\"")
          ),
          List("- root class: \"long\"")
        ),
        Long
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, LongType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, LongType),
          LongType,
          List("- root class: \"long\"")
        ),
        List("- root class: \"long\"")
      ),
      Long
    )
  ),
  None,
  false,
  true
)

In [19]:
val t1 = System.nanoTime

train
    .select($"content_id", to_month($"timestamp").as("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration1 = (System.nanoTime - t1) / 1e9d

24/09/09 20:22:11 INFO CodeGenerator: Code generated in 39.443455 ms
24/09/09 20:22:11 INFO DAGScheduler: Registering RDD 35 (show at cell19.sc:6) as input to shuffle 1
24/09/09 20:22:11 INFO DAGScheduler: Got map stage job 7 (show at cell19.sc:6) with 44 output partitions
24/09/09 20:22:11 INFO DAGScheduler: Final stage: ShuffleMapStage 8 (show at cell19.sc:6)
24/09/09 20:22:11 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:22:11 INFO DAGScheduler: Missing parents: List()
24/09/09 20:22:11 INFO DAGScheduler: Submitting ShuffleMapStage 8 (MapPartitionsRDD[35] at show at cell19.sc:6), which has no missing parents
24/09/09 20:22:11 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 54.6 KiB, free 2.2 GiB)
24/09/09 20:22:11 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 24.4 KiB, free 2.2 GiB)
24/09/09 20:22:11 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on ubuntu:41801 (size: 24.4 KiB, free:

+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|0.4460020768431983|
|      4818|0.5068813295247988|
|      5518|0.6536082474226804|
|     13285| 2.725925925925926|
|     12027|2.2696177062374248|
|       833|0.7334516865330967|
|      9427| 0.652203699474766|
|       496|0.6270834569072898|
|      5156|0.5233297393761386|
|      6336|0.6127002288329519|
|      2866|0.9061162849232318|
|      1959|1.0353606789250354|
|      7982|0.6966064981949458|
|     23336| 0.564428149870339|
|       148| 0.602506180206291|
|      1342|0.7789912124752375|
|      1088|0.7588338484671289|
|      1580|0.7253544345532477|
|       471|0.8491909385113269|
|      2122|0.8964813870474247|
+----------+------------------+
only showing top 20 rows



t1: Long = 102933075036219L
duration1: Double = 4.418106916

In [20]:
val t2 = System.nanoTime

train
    .select($"content_id", ($"timestamp" / 31536000000L * 12).alias("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration2 = (System.nanoTime - t2) / 1e9d

24/09/09 20:22:16 INFO CodeGenerator: Code generated in 24.994573 ms
24/09/09 20:22:16 INFO DAGScheduler: Registering RDD 43 (show at cell20.sc:6) as input to shuffle 2
24/09/09 20:22:16 INFO DAGScheduler: Got map stage job 9 (show at cell20.sc:6) with 44 output partitions
24/09/09 20:22:16 INFO DAGScheduler: Final stage: ShuffleMapStage 11 (show at cell20.sc:6)
24/09/09 20:22:16 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:22:16 INFO DAGScheduler: Missing parents: List()
24/09/09 20:22:16 INFO DAGScheduler: Submitting ShuffleMapStage 11 (MapPartitionsRDD[43] at show at cell20.sc:6), which has no missing parents
24/09/09 20:22:16 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 49.5 KiB, free 2.2 GiB)
24/09/09 20:22:16 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 22.2 KiB, free 2.2 GiB)
24/09/09 20:22:16 INFO BlockManagerInfo: Removed broadcast_11_piece0 on ubuntu:41801 in memory (size: 25.5 KiB, f

+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|2.2218736221822732|
|      4818|2.3372134952525556|
|      5518| 3.029691836679273|
|     13285| 5.964229418381344|
|     12027|5.9773868010962765|
|       833|  3.25216382900438|
|      9427| 3.108803192689145|
|       496|2.9023706505861853|
|      5156|2.6012427887203198|
|      6336|2.7640873674707946|
|      2866|3.8766860651262456|
|      1959|3.8461177532795423|
|      7982|3.0568602514634398|
|     23336| 2.773484159471813|
|       148| 2.579153999078694|
|      1342|  3.16126199595755|
|      1088| 3.545207472523368|
|      1580|3.2343945900405027|
|       471| 3.760585843300882|
|      2122|3.7522109521798543|
+----------+------------------+
only showing top 20 rows



t2: Long = 102937861017798L
duration2: Double = 3.278641194

In [21]:
println(s"duration1 = $duration1\nduration2 = $duration2\nduration1 - duration2 = ${duration1 - duration2}")

duration1 = 4.418106916
duration2 = 3.278641194
duration1 - duration2 = 1.1394657220000002
